In [1]:
# La Cam da OpenCV deve essere usate tramite GStreamer con un comando dedicato
# https://stackoverflow.com/questions/64272731/open-cv-shows-green-screen-on-jetson-nano
#
# Eseguire :
# sudo nvpmodel -m 0
# sudo jetson_clocks
# per portare a 4 le CPU operative. OpenCV su NVIDIA Jetson Nano non è ottimizzato per usare la GPU
# Il codice gira tutto su CPU con una certa lentezza.
#
import cv2
import numpy as np
import math
import os
from matplotlib import pyplot as plt
from IPython.display import clear_output, display, Image
import ipywidgets
import time

%matplotlib inline


In [2]:
# make_chunks function creats chunks.
# inputs -- EdgeArray and the size_of_chunk to create.
# output -- yield successive n-sized chunks.
def make_chunks(EdgeArray, size_of_chunk):
    chunks = []
    for i in range(0, len(EdgeArray), size_of_chunk):
        chunks.append(EdgeArray[i:i + size_of_chunk])
    return chunks


In [3]:
#
# https://github.com/JetsonHacksNano/CSI-Camera/blob/master/simple_camera.py
#

# MIT License
# Copyright (c) 2019-2022 JetsonHacks

# Using a CSI camera (such as the Raspberry Pi Version 2) connected to a
# NVIDIA Jetson Nano Developer Kit using OpenCV
# Drivers for the camera and OpenCV are included in the base image

import cv2

""" 
gstreamer_pipeline returns a GStreamer pipeline for capturing from the CSI camera
Flip the image by setting the flip_method (most common values: 0 and 2)
display_width and display_height determine the size of each camera pane in the window on the screen
Default 1920x1080 displayd in a 1/4 size window
"""

def gstreamer_pipeline(
    sensor_id=0,
    capture_width=640,
    capture_height=480,
    display_width=640,
    display_height=480,
    framerate=20,
    flip_method=0,
):
    return (
        "nvarguscamerasrc sensor-id=%d !"
        "video/x-raw(memory:NVMM), width=(int)%d, height=(int)%d, framerate=(fraction)%d/1 ! "
        "nvvidconv flip-method=%d ! "
        "video/x-raw, width=(int)%d, height=(int)%d, format=(string)BGRx ! "
        "videoconvert ! "
        "video/x-raw, format=(string)BGR ! appsink"
        % (
            sensor_id,
            capture_width,
            capture_height,
            framerate,
            flip_method,
            display_width,
            display_height,
        )
    )

In [5]:
#
# To display images to verify. Set to zero if not intrested in displaying images.
testmode = 1  

# Code to draw edge representation.

# cap = cv2.VideoCapture(0)  # Creating object to capturing frame from inbult camera(0) or the external camera(1)
cap = cv2.VideoCapture(gstreamer_pipeline(flip_method=0), cv2.CAP_GSTREAMER)

if testmode == 1:
    display_handle=display(None, display_id=True)
    navigation_w = ipywidgets.Image(format='jpeg')

StepSize = 5

try:
    while (1):
        _, frame = cap.read()  # Reading the frame from the object.
        img_navigation = frame.copy()  # Copy of frame which will be used for indicating direction of navigation.

            # Code to display the results
        if testmode == 1:
            font = cv2.FONT_HERSHEY_SIMPLEX
            navigation = cv2.putText(img_navigation, "Ciao", (275, 50), font, 1, (0, 0, 255), 2, cv2.LINE_AA)

            clear_output(wait=True)            
            lines, columns, _ =  navigation.shape
            navigation = cv2.resize(navigation, (int(columns/4), int(lines/4)))
            navigation_w.value =cv2.imencode('.jpeg', navigation)[1].tobytes()
            display(navigation_w)
            
            #time.sleep(0.5)
            
except KeyboardInterrupt:
    pass
finally:
    cap.release()
    display_handle.update(None)    

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

In [ ]:
_, frame = cap.read()
frame

In [ ]:
cv2.destroyAllWindows
cap.release()